## Word Embedding

In [ ]:
import os
import numpy as np
from docx import Document
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [ ]:
def pad_embeddings(a, b):
    target_length = max(len(a), len(b))
    a = np.pad(a, (0, target_length - len(a)), 'constant')
    b = np.pad(b, (0, target_length - len(b)), 'constant')
    return a, b

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
baseline_text = """
In the picture, a busy scene in a kitchen is depicted. Here are the details:

1. **People**:
   - There is a woman, likely the mother, standing by the sink washing dishes. She is holding a plate and a dish towel.
   - A young boy is standing on a stool, reaching into an upper cabinet where there is a jar labeled "COOKIE JAR." He has one hand in the jar and appears to be retrieving a cookie.
   - A young girl stands next to the boy, reaching up as if to help or receive a cookie from him.

2. **Actions**:
   - The woman is washing dishes, but the sink is overflowing with water, which is spilling onto the floor.
   - The boy is taking cookies from the jar.
   - The girl is either reaching to help the boy or to receive a cookie from him.

3. **Objects**:
   - **Cookie jar**: Clearly labeled and located in an upper cabinet.
   - **Stool**: The boy is standing on it to reach the cookie jar.
   - **Sink**: Overflowing with water, indicating a possible plumbing issue or neglect due to distraction.
   - **Dishes**: The woman is holding one, and there are a few other dishes visible on the counter.

4. **Environment**:
   - **Kitchen**: The setting is a typical kitchen with cabinets, a counter, a sink, and a window.
   - **Window**: Through the window, a scene of a tree or bush is visible, suggesting it might be a backyard or garden view.
   - **Curtains**: The window has curtains that are pulled back.

The overall scene shows a mix of normal daily activity (dishwashing) with a bit of mischief (children reaching for cookies) and a potential mishap (overflowing sink).
"""

In [ ]:
def read_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return full_text

In [ ]:
full_text_list = read_docx("./data/P16_S1_check.docx")

In [ ]:
full_text_list


In [ ]:
t1_start_index = 0
t1_end_index = 0
for paragraph in full_text_list:
    if "storytelling" in paragraph.lower():
        break
    t1_start_index += 1
for paragraph in full_text_list:
    if "t2" in paragraph.lower():
        break
    t1_end_index += 1

t2_start_index = 0
t2_end_index = len(full_text_list) - 1
for paragraph in full_text_list:
    if "different game" in paragraph.lower():
        break
    t2_start_index += 1

In [ ]:
print(t1_start_index, t1_end_index, t2_start_index, t2_end_index)

print(full_text_list[t1_start_index])
print(full_text_list[t1_end_index])

print(full_text_list[t2_start_index])
print(full_text_list[t2_end_index])

In [ ]:
T1 = (full_text_list[t1_start_index:t1_end_index])

In [ ]:
T2 = (full_text_list[t2_start_index:t2_end_index])

In [ ]:
T1_processed = list(
    filter(lambda x: not ((x == "") or ("speaker 0" in x.lower()) or (x is None) or ("first prompt" in x.lower())),
           T1))
T2_processed = list(
    filter(lambda x: not ((x == "") or ("speaker 0" in x.lower()) or (x is None) or ("first prompt" in x.lower())),
           T2))

In [ ]:
for i in range(0, len(T1_processed)):
    T1_processed[i] = T1_processed[i].replace("Speaker 1: ", "")
for i in range(0, len(T2_processed)):
    T2_processed[i] = T2_processed[i].replace("Speaker 1: ", "")

In [ ]:
T1_processed

In [ ]:
"".join(T1_processed)

In [ ]:
"".join(T2_processed)

In [ ]:
test_text = "".join(T1_processed)

In [ ]:
baseline_text_emb = get_embedding(baseline_text)
test_text_emb = get_embedding(test_text)

In [ ]:
baseline_text_emb, test_text_emb = pad_embeddings(baseline_text_emb, test_text_emb)

In [ ]:
cosine_similarity(baseline_text_emb, test_text_emb)

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("intfloat/e5-mistral-7b-instruct")
embeddings = model.encode([
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium.",
])
similarities = model.similarity(embeddings, embeddings)